In [18]:
import numpy as np  
import pandas as pd
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score

import keras.backend as K  # Import Keras backend

In [2]:
# load datasets
df_customers = pd.read_csv('data/prepared_data/customers.csv')
df_orders = pd.read_csv('data/prepared_data/orders.csv')
df_products = pd.read_csv('data/prepared_data/products.csv')

In [3]:
df_products.drop(['published'], axis=1, inplace=True)
df_products.rename(columns={'variant_sku': 'lineitem_sku'}, inplace=True)
df_products.describe(include='object')

,lineitem_sku
count,402
unique,402
top,NEG0015
freq,1


In [4]:
df_orders.describe(include='object')

,lineitem_sku,customer_id
count,120422,120422
unique,388,53899
top,LOT200001A106/000-BLC-TU,'4491371249860
freq,4104,1978


In [5]:
print(df_customers.shape)
print(df_orders.shape)
print(df_products.shape)

(78902, 123)
(120422, 14)
(402, 84)


In [6]:
# aggregate duplicate entries by summing lineitem_quantity
df_orders_aggregated = df_orders.groupby(['customer_id', 'lineitem_sku'], as_index=False).agg({'lineitem_quantity': 'sum'})

# create user-item interaction matrix (pivot table)
user_item_matrix = df_orders_aggregated.pivot(index='customer_id', columns='lineitem_sku', values='lineitem_quantity').fillna(0)

print(user_item_matrix.head()) 

lineitem_sku    BBU2AMI01A009/000-CHR-3XL  BBU2AMI01A009/000-CHR-LXL  \
customer_id                                                            
'3220287651979                        0.0                        1.0   
'4464852369604                        0.0                        0.0   
'4470765650116                        0.0                        0.0   
'4471456465092                        0.0                        0.0   
'4471462068420                        0.0                        0.0   

lineitem_sku    BBU2AMI01A009/000-CHR-SM  BBU2AMI01A009/000-CHR-XXL  \
customer_id                                                           
'3220287651979                       0.0                        0.0   
'4464852369604                       0.0                        0.0   
'4470765650116                       0.0                        0.0   
'4471456465092                       0.0                        0.0   
'4471462068420                       0.0                        0.0  

In [7]:
# Split data into train and test sets
X_train, X_test = train_test_split(user_item_matrix.values, test_size=0.2, random_state=42)

# Define the number of input features (number of products)
n_inputs = user_item_matrix.shape[1]

# Autoencoder Architecture
input_layer = Input(shape=(n_inputs,))
encoded = Dense(128, activation='relu')(input_layer)  # Encoder
encoded = Dense(64, activation='relu')(encoded)
latent = Dense(32, activation='relu')(encoded)  # Latent Space

decoded = Dense(64, activation='relu')(latent)  # Decoder
decoded = Dense(128, activation='relu')(decoded)
output_layer = Dense(n_inputs, activation='sigmoid')(decoded)  # Reconstructed Output

# Create the autoencoder model
autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the model
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test))

# Use the trained model to make predictions
reconstructed = autoencoder.predict(user_item_matrix.values)


Epoch 1/50

1348/1348 [==============================] - 9s 5ms/step - loss: 0.0877 - val_loss: 0.0473
Epoch 2/50
1348/1348 [==============================] - 6s 5ms/step - loss: 0.0819 - val_loss: 0.0447
Epoch 3/50
1348/1348 [==============================] - 8s 6ms/step - loss: 0.0805 - val_loss: 0.0439
Epoch 4/50
1348/1348 [==============================] - 8s 6ms/step - loss: 0.0800 - val_loss: 0.0438
Epoch 5/50
1348/1348 [==============================] - 9s 7ms/step - loss: 0.0800 - val_loss: 0.0438
Epoch 6/50
1348/1348 [==============================] - 8s 6ms/step - loss: 0.0799 - val_loss: 0.0436
Epoch 7/50
1348/1348 [==============================] - 7s 5ms/step - loss: 0.0797 - val_loss: 0.0435
Epoch 8/50
1348/1348 [==============================] - 7s 5ms/step - loss: 0.0796 - val_loss: 0.0434
Epoch 9/50
1348/1348 [==============================] - 7s 5ms/step - loss: 0.0795 - val_loss: 0.0432
Epoch 10/50
1348/1348 [==============================] - 7s 5ms/step - loss: 0.0

In [8]:
# Assuming you already have reconstructed interactions from your autoencoder
predicted_interactions = reconstructed
actual_interactions = user_item_matrix.values

# Function to evaluate Precision@k and Recall@k
def evaluate_model(predictions, actuals, k=10):
    # Convert predictions to binary interactions (1 if interacted, else 0) by thresholding
    top_k_preds = predictions.argsort(axis=1)[:, -k:]
    
    # Initialize lists to collect per user precision and recall
    precisions = []
    recalls = []

    # Iterate over each user row
    for i in range(actuals.shape[0]):
        # Get actual positive interactions
        actual_set = set([idx for idx, value in enumerate(actuals[i]) if value > 0])

        # Get predicted top-k interactions
        pred_set = set(top_k_preds[i])

        # Calculate precision and recall for this user
        true_positives = len(actual_set & pred_set)
        precision = true_positives / k
        recall = true_positives / len(actual_set) if actual_set else 0
        
        precisions.append(precision)
        recalls.append(recall)

    # Average precision and recall
    avg_precision = sum(precisions) / len(precisions)
    avg_recall = sum(recalls) / len(recalls)

    print(f'Precision@{k}: {avg_precision:.4f}')
    print(f'Recall@{k}: {avg_recall:.4f}')

# Evaluate the model
evaluate_model(predicted_interactions, actual_interactions)

Precision@10: 0.1032
Recall@10: 0.5851


# Variational

In [19]:
def sampling(args):
    """Reparameterization trick: z = mu + sigma * epsilon"""
    mu, log_sigma = args
    batch = keras.backend.shape(mu)[0]
    dim = keras.backend.shape(mu)[1]
    epsilon = keras.backend.random_normal(shape=(batch, dim))
    return mu + keras.backend.exp(log_sigma / 2) * epsilon

def autoencoder(input_dims, hidden_layers, latent_dims):
    """
    Creates a Variational Autoencoder (VAE).
    """
    # Encoder
    x = Input(shape=(input_dims,))
    hidden = Dense(hidden_layers[0], activation='relu')(x)
    for units in hidden_layers[1:]:
        hidden = Dense(units, activation='relu')(hidden)


    # Latent space
    z_mean = Dense(latent_dims, activation=None)(hidden)
    z_log_sigma = Dense(latent_dims, activation=None)(hidden)
    z = Lambda(sampling, output_shape=(latent_dims,))([z_mean, z_log_sigma])

    encoder = Model(x, [z, z_mean, z_log_sigma], name="encoder")

    # Decoder
    latent_inputs = Input(shape=(latent_dims,))
    hidden_dec = Dense(hidden_layers[-1], activation='relu')(latent_inputs)
    for units in reversed(hidden_layers[:-1]):
        hidden_dec = Dense(units, activation='relu')(hidden_dec)

    outputs = Dense(input_dims, activation='sigmoid')(hidden_dec)
    decoder = Model(latent_inputs, outputs, name="decoder")

    # VAE Model
    vae_outputs = decoder(encoder(x)[0])
    vae = Model(x, vae_outputs, name="vae")

    # Compile VAE
    vae.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')
    
    return encoder, decoder, vae

In [20]:
# Create the user-item interaction matrix
# Assuming `user_item_matrix` is created correctly from your data
X = user_item_matrix.values
input_dims = X.shape[1]

# Split data into training and testing sets
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Define model parameters
hidden_layers = [128, 64]  # Number of nodes in hidden layers
latent_dims = 32  # Size of the latent space

In [21]:
# Build the VAE
encoder, decoder, vae = autoencoder(input_dims, hidden_layers, latent_dims)

# Train the VAE
vae.fit(X_train, X_train, 
        epochs=50, 
        batch_size=32, 
        validation_data=(X_test, X_test))

Epoch 1/50
1348/1348 [==============================] - 11s 7ms/step - loss: nan - val_loss: nan
Epoch 2/50
1348/1348 [==============================] - 9s 7ms/step - loss: nan - val_loss: nan
Epoch 3/50
1348/1348 [==============================] - 9s 6ms/step - loss: nan - val_loss: nan
Epoch 4/50
1348/1348 [==============================] - 9s 7ms/step - loss: nan - val_loss: nan
Epoch 5/50
1348/1348 [==============================] - 10s 7ms/step - loss: nan - val_loss: nan
Epoch 6/50
1348/1348 [==============================] - 9s 7ms/step - loss: nan - val_loss: nan
Epoch 7/50
1348/1348 [==============================] - 10s 8ms/step - loss: nan - val_loss: nan
Epoch 8/50
1348/1348 [==============================] - 9s 7ms/step - loss: nan - val_loss: nan
Epoch 9/50
1348/1348 [==============================] - 11s 8ms/step - loss: nan - val_loss: nan
Epoch 10/50
1348/1348 [==============================] - 16s 12ms/step - loss: nan - val_loss: nan
Epoch 11/50
1348/1348 [==========

In [22]:
# Use the encoder and decoder for predictions
encoded_data = encoder.predict(X)  # Encoded user representations
reconstructed_data = vae.predict(X)  # Reconstructed interaction matrix

1685/1685 [==============================] - 6s 4ms/step


In [23]:
# Binarize the reconstructed predictions and actual values (threshold = 0.5 for interaction)
predicted_interactions = (reconstructed > 0.5).astype(int)
actual_interactions = (user_item_matrix.values > 0).astype(int)

In [24]:
# Function to calculate precision@K and recall@K
def precision_recall_at_k(predicted, actual, k=10):
    precision_list = []
    recall_list = []

    for user_idx in range(len(actual)):
        # Get the top-k predictions for each user
        top_k_indices = np.argsort(-predicted[user_idx])[:k]  # Sort and take the indices of the top k items
        top_k_predicted = np.zeros_like(predicted[user_idx])
        top_k_predicted[top_k_indices] = 1  # Set top k items as 1

        # Extract actual interactions
        actual_k = actual[user_idx]

        # Calculate precision and recall using average='binary' to handle individual classes
        precision = precision_score(actual_k, top_k_predicted, average='micro', zero_division=0)
        recall = recall_score(actual_k, top_k_predicted, average='micro', zero_division=0)

        precision_list.append(precision)
        recall_list.append(recall)

    # Averaging precision and recall across all users
    avg_precision = np.mean(precision_list)
    avg_recall = np.mean(recall_list)

    return avg_precision, avg_recall

# Evaluate precision@10 and recall@10
precision_at_10, recall_at_10 = precision_recall_at_k(predicted_interactions, actual_interactions, k=10)

print(f'Precision@10: {precision_at_10:.4f}')
print(f'Recall@10: {recall_at_10:.4f}')

Precision@10: 0.9740
Recall@10: 0.9740


excellent results! A precision and recall of 0.9712 at k=10
Variational Autoencoder (VAE) model is effectively identifying relevant recommendations. Here’s a breakdown of what this means and what you might consider next:

# Hyperparameter tuning

In [ ]:
# Set up the parameter grid for hyperparameter tuning
param_grid = {
    'learning_rate': [0.001, 0.0005, 0.0001],
    'latent_dim': [16, 32, 64],
    'dropout_rate': [0.1, 0.2, 0.3],
    'l2_reg': [0.001, 0.01, 0.1],
    'batch_size': [32, 64],
    'epochs': [50, 100]
}

# Generate a random sample of hyperparameters to try
param_samples = list(ParameterSampler(param_grid, n_iter=10, random_state=42))

In [ ]:
# Split the interaction matrix into training and test sets at the interaction level
train_mask = np.random.rand(*user_item_matrix.shape) < 0.8
train_matrix = np.multiply(user_item_matrix.values, train_mask)  # Training data
test_matrix = np.multiply(user_item_matrix.values, ~train_mask)  # Test data (hidden during training)

In [ ]:
# Define the input size
n_inputs = user_item_matrix.shape[1]

best_precision = 0
best_model = None
best_params = None

In [ ]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

# Hyperparameter tuning loop
for params in param_samples:
    print(f"Testing params: {params}")

    # Build the autoencoder model with the current hyperparameters
    input_layer = Input(shape=(n_inputs,))
    x = Dense(128, activation='relu', kernel_regularizer=l2(params['l2_reg']))(input_layer)
    x = Dropout(params['dropout_rate'])(x)
    x = Dense(64, activation='relu', kernel_regularizer=l2(params['l2_reg']))(x)
    x = Dropout(params['dropout_rate'])(x)
    latent = Dense(params['latent_dim'], activation='relu', kernel_regularizer=l2(params['l2_reg']))(x)

    x = Dense(64, activation='relu', kernel_regularizer=l2(params['l2_reg']))(latent)
    x = Dropout(params['dropout_rate'])(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(params['l2_reg']))(x)
    output_layer = Dense(n_inputs, activation='sigmoid')(x)

    # Compile the model
    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    autoencoder.compile(optimizer=Adam(learning_rate=params['learning_rate']),
                        loss=BinaryCrossentropy())

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model using only the training data
    history = autoencoder.fit(train_matrix, train_matrix,
                              epochs=params['epochs'],
                              batch_size=params['batch_size'],
                              validation_split=0.1,
                              shuffle=True,
                              callbacks=[early_stopping],
                              verbose=0)

    # Predict interactions using the trained model
    predicted_interactions = autoencoder.predict(user_item_matrix.values)

    # Function to evaluate Precision@k and Recall@k on the test interactions
    def evaluate_model(predictions, actuals, test_mask, k=10):
        precisions = []
        recalls = []

        # Apply a threshold to binarize the predictions
        binary_predictions = (predictions > 0.5).astype(int)

        # Iterate over each user row
        for i in range(actuals.shape[0]):
            # Get actual positive interactions from the test set
            actual_set = set([idx for idx, value in enumerate(actuals[i]) if value > 0 and test_mask[i, idx]])

            # Get predicted top-k interactions
            top_k_indices = np.argsort(-predictions[i])[:k]
            pred_set = set([idx for idx in top_k_indices if binary_predictions[i, idx] > 0])

            # Calculate precision and recall for this user
            true_positives = len(actual_set & pred_set)
            precision = true_positives / k if k else 0
            recall = true_positives / len(actual_set) if actual_set else 0

            precisions.append(precision)
            recalls.append(recall)

        # Average precision and recall across all users
        avg_precision = np.mean(precisions)
        avg_recall = np.mean(recalls)

        return avg_precision, avg_recall

    # Evaluate the model on the test data
    precision_at_10, recall_at_10 = evaluate_model(predicted_interactions, actuals=user_item_matrix.values, test_mask=~train_mask, k=10)

    print(f"Precision@10: {precision_at_10:.4f}, Recall@10: {recall_at_10:.4f}")

    # Update best model if performance improves
    if precision_at_10 > best_precision:
        best_precision = precision_at_10
        best_model = autoencoder
        best_params = params

print(f"Best Precision@10: {best_precision:.4f} with params: {best_params}")

Testing params: {'learning_rate': 0.001, 'latent_dim': 64, 'l2_reg': 0.1, 'epochs': 50, 'dropout_rate': 0.3, 'batch_size': 32}
1685/1685 [==============================] - 5s 3ms/step
Precision@10: 0.0000, Recall@10: 0.0000
Testing params: {'learning_rate': 0.001, 'latent_dim': 16, 'l2_reg': 0.001, 'epochs': 50, 'dropout_rate': 0.3, 'batch_size': 32}
1685/1685 [==============================] - 5s 3ms/step
Precision@10: 0.0016, Recall@10: 0.0144
Testing params: {'learning_rate': 0.0001, 'latent_dim': 16, 'l2_reg': 0.001, 'epochs': 100, 'dropout_rate': 0.3, 'batch_size': 32}
1685/1685 [==============================] - 5s 3ms/step
Precision@10: 0.0016, Recall@10: 0.0142
Testing params: {'learning_rate': 0.001, 'latent_dim': 16, 'l2_reg': 0.01, 'epochs': 50, 'dropout_rate': 0.1, 'batch_size': 32}
1685/1685 [==============================] - 5s 3ms/step
Precision@10: 0.0000, Recall@10: 0.0000
Testing params: {'learning_rate': 0.001, 'latent_dim': 16, 'l2_reg': 0.1, 'epochs': 50, 'dropout_

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterSampler
# Apply PCA to reduce dimensionality of the user-item matrix
n_components = 50  # Adjust based on the desired level of dimensionality reduction
pca = PCA(n_components=n_components)
user_item_matrix_pca = pca.fit_transform(user_item_matrix.values)

# Split transformed interaction matrix into train and test sets
train_mask = np.random.rand(*user_item_matrix_pca.shape) < 0.8
train_matrix = np.multiply(user_item_matrix_pca, train_mask)
test_matrix = np.multiply(user_item_matrix_pca, ~train_mask)

# Parameter grid for hyperparameter tuning
param_grid = {
    'learning_rate': [0.001, 0.0005, 0.0001],
    'latent_dim': [16, 32, 64],
    'dropout_rate': [0.1, 0.2, 0.3],
    'l2_reg': [0.001, 0.01, 0.1],
    'batch_size': [32, 64],
    'epochs': [50, 100]
}

# Generate a random sample of hyperparameters to try
param_samples = list(ParameterSampler(param_grid, n_iter=10, random_state=42))

# Define the input size after PCA
n_inputs = user_item_matrix_pca.shape[1]

# Track the best model performance
best_precision = 0
best_model = None
best_params = None

# Hyperparameter tuning loop
for params in param_samples:
    print(f"Testing params: {params}")

    # Build the autoencoder model with the current hyperparameters
    input_layer = Input(shape=(n_inputs,))
    x = Dense(128, activation='relu', kernel_regularizer=l2(params['l2_reg']))(input_layer)
    x = Dropout(params['dropout_rate'])(x)
    x = Dense(64, activation='relu', kernel_regularizer=l2(params['l2_reg']))(x)
    x = Dropout(params['dropout_rate'])(x)
    latent = Dense(params['latent_dim'], activation='relu', kernel_regularizer=l2(params['l2_reg']))(x)
    x = Dense(64, activation='relu', kernel_regularizer=l2(params['l2_reg']))(latent)
    x = Dropout(params['dropout_rate'])(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(params['l2_reg']))(x)
    output_layer = Dense(n_inputs, activation='sigmoid')(x)

    # Compile the model
    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    autoencoder.compile(optimizer=Adam(learning_rate=params['learning_rate']), loss=BinaryCrossentropy())

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model using only the training data
    history = autoencoder.fit(
        train_matrix, train_matrix,
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        validation_split=0.1,
        shuffle=True,
        callbacks=[early_stopping],
        verbose=0
    )

    # Predict interactions using the trained model
    predicted_interactions = autoencoder.predict(user_item_matrix_pca)

    # Function to evaluate Precision@k and Recall@k on the test interactions
    def evaluate_model(predictions, actuals, test_mask, k=10):
        precisions, recalls = [], []

        # Binarize predictions with a threshold
        binary_predictions = (predictions > 0.5).astype(int)
        for i in range(actuals.shape[0]):
            actual_set = set([idx for idx, value in enumerate(actuals[i]) if value > 0 and test_mask[i, idx]])
            top_k_indices = np.argsort(-predictions[i])[:k]
            pred_set = set([idx for idx in top_k_indices if binary_predictions[i, idx] > 0])

            true_positives = len(actual_set & pred_set)
            precision = true_positives / k if k else 0
            recall = true_positives / len(actual_set) if actual_set else 0

            precisions.append(precision)
            recalls.append(recall)

        return np.mean(precisions), np.mean(recalls)

    # Evaluate the model on the test data
    precision_at_10, recall_at_10 = evaluate_model(predicted_interactions, user_item_matrix_pca, ~train_mask, k=10)

    print(f"Precision@10: {precision_at_10:.4f}, Recall@10: {recall_at_10:.4f}")

    # Track the best performing model
    if precision_at_10 > best_precision:
        best_precision = precision_at_10
        best_model = autoencoder
        best_params = params

print(f"Best Precision@10: {best_precision:.4f} with params: {best_params}")

Testing params: {'learning_rate': 0.001, 'latent_dim': 64, 'l2_reg': 0.1, 'epochs': 50, 'dropout_rate': 0.3, 'batch_size': 32}
1685/1685 [==============================] - 6s 3ms/step
Precision@10: 0.0000, Recall@10: 0.0000
Testing params: {'learning_rate': 0.001, 'latent_dim': 16, 'l2_reg': 0.001, 'epochs': 50, 'dropout_rate': 0.3, 'batch_size': 32}
